In [1]:
using LinearAlgebra
using PyPlot
using ForwardDiff
using MatrixCalculus

In [2]:
#Model parameters
g = 9.81 #m/s^2
m = 1.0 #kg 
ℓ = 0.3 #meters

#Thrust and gimbal limits
umin = [0.0; 0.0]
umax = [0.6*m*g; 0.6*m*g]

h = 0.025 #time step (50 Hz)

0.025

In [3]:
#Planar Quadrotor Dynamics
function quad_dynamics(x,u)
    J = 0.2*m*ℓ*ℓ
    
    θ = x[3]
    
    ẍ = (1/m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/m)*(u[1] + u[2])*cos(θ) - g
    θ̈ = (1/J)*(ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end

quad_dynamics (generic function with 1 method)

In [4]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
#First derivatives of dynamics

function dfdx(x,u)
    return ForwardDiff.jacobian(dx->quad_dynamics_rk4(dx,u),x)
end

function dfdu(x,u)
    return ForwardDiff.jacobian(du->quad_dynamics_rk4(x,du),u)
end

dfdu (generic function with 1 method)

In [6]:
#Second derivatives of dynamics

function dAdx(x,u)
    return ForwardDiff.jacobian(dx->vec(dfdx(dx,u)),x)
end

function dBdx(x,u)
    return ForwardDiff.jacobian(dx->vec(dfdu(dx,u)),x)
end

function dAdu(x,u)
    return ForwardDiff.jacobian(du->vec(dfdx(x,du)),u)
end

function dBdu(x,u)
    return ForwardDiff.jacobian(du->vec(dfdu(x,du)),u)
end

dBdu (generic function with 1 method)

In [7]:
Nx = 6     # number of state
Nu = 2     # number of controls
Tfinal = 1.5 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [8]:
# Cost weights
Q = Diagonal([ones(3); 0.1*ones(3)]);
R = Array(.01*I(Nu));
Qn = Array(100.0*I(Nx));

In [24]:
#Reference trajectory for flip
x1ref = [LinRange(-3,0,20); zeros(20); LinRange(0,3,21)]
x2ref = [ones(20); LinRange(1,3,10); LinRange(3,1,10); ones(21)]
θref = [zeros(20); LinRange(0,-2*pi,20); -2*pi*ones(21)]
v1ref = [6.0*ones(20); zeros(20); 6.0*ones(21)]
v2ref = [zeros(20); 8.0*ones(10); -8.0*ones(10); zeros(21)]
ωref = [zeros(20); -4*pi*ones(20); zeros(21)]
xref = [x1ref'; x2ref'; θref'; v1ref'; v2ref'; ωref']

6×61 Array{Float64,2}:
 -3.0  -2.84211  -2.68421  -2.52632  …   2.7       2.85      3.0
  1.0   1.0       1.0       1.0          1.0       1.0       1.0
  0.0   0.0       0.0       0.0         -6.28319  -6.28319  -6.28319
  6.0   6.0       6.0       6.0          6.0       6.0       6.0
  0.0   0.0       0.0       0.0          0.0       0.0       0.0
  0.0   0.0       0.0       0.0      …   0.0       0.0       0.0

In [10]:
#Reference trajectory for straight line
x1ref = Array(LinRange(-2,2,101))
x2ref = Array(ones(101))
θref = Array(zeros(101))
v1ref = Array((4.0/3.0)*ones(101))
v2ref = Array(zeros(101))
ωref = Array(zeros(101))
xref = [x1ref'; x2ref'; θref'; v1ref'; v2ref'; ωref']

6×101 Array{Float64,2}:
 -2.0      -1.96     -1.92     …  1.88     1.92     1.96     2.0
  1.0       1.0       1.0         1.0      1.0      1.0      1.0
  0.0       0.0       0.0         0.0      0.0      0.0      0.0
  1.33333   1.33333   1.33333     1.33333  1.33333  1.33333  1.33333
  0.0       0.0       0.0         0.0      0.0      0.0      0.0
  0.0       0.0       0.0      …  0.0      0.0      0.0      0.0

In [11]:
function stage_cost(x,u,k)
    return 0.5*(x-xref[:,k])'*Q*(x-xref[:,k]) + 0.5*(u-uhover)'*R*(u-uhover)
    #return 0.5*(x-xgoal)'*Q*(x-xgoal) + 0.5*(u-uhover)'*R*(u-uhover)
end

stage_cost (generic function with 1 method)

In [12]:
function terminal_cost(x)
    return 0.5*(x-xref[:,Nt])'*Qn*(x-xref[:,Nt])
    #return 0.5*(x-xgoal)'*Qn*(x-xgoal)
end

terminal_cost (generic function with 1 method)

In [13]:
function cost(xtraj,utraj)
    J = 0
    for k = 1:(Nt-1)
        J += stage_cost(xtraj[:,k],utraj[:,k],k)
    end
    J += terminal_cost(xtraj[:,Nt])
    return J
end

cost (generic function with 1 method)

In [57]:
#Initial guess trajectory (hover)
x0 = [-2.0; 1.0; 0; 0; 0; 0]
xgoal = [2.0; 1.0; 0; 0; 0; 0]
uhover = [0.5*m*g; 0.5*m*g]
xtraj = kron(ones(1,Nt), x0)
utraj = kron(ones(1,Nt-1), uhover)
J = cost(xtraj,utraj)

6037.32751887312

In [58]:
#DDP Algorithm
p = zeros(Nx,Nt)
P = zeros(Nx,Nx,Nt)
j = ones(Nu,Nt-1)
K = zeros(Nu,Nx,Nt-1)
ΔJ = 0.0

iter = 0
while maximum(abs.(j[:])) > 1e-2
    iter += 1
    
    p = zeros(Nx,Nt)
    P = zeros(Nx,Nx,Nt)
    j = zeros(Nu,Nt-1)
    K = zeros(Nu,Nx,Nt-1)
    ΔJ = 0.0

    p[:,Nt] = ForwardDiff.gradient(terminal_cost,xtraj[:,Nt])
    P[:,:,Nt] = ForwardDiff.hessian(terminal_cost,xtraj[:,Nt])
    
    #Backward Pass
    for k = (Nt-1):-1:1
        #Calculate derivatives
        q = ForwardDiff.gradient(x->stage_cost(x,utraj[:,k],k),xtraj[:,k])
        Q = ForwardDiff.hessian(x->stage_cost(x,utraj[:,k],k),xtraj[:,k])
    
        r = ForwardDiff.gradient(u->stage_cost(xtraj[:,k],u,k),utraj[:,k])
        R = ForwardDiff.hessian(u->stage_cost(xtraj[:,k],u,k),utraj[:,k])
    
        A = dfdx(xtraj[:,k],utraj[:,k])
        B = dfdu(xtraj[:,k],utraj[:,k])
    
        Ax = dAdx(xtraj[:,k],utraj[:,k])
        Bx = dBdx(xtraj[:,k],utraj[:,k])
        Au = dAdu(xtraj[:,k],utraj[:,k])
        Bu = dBdu(xtraj[:,k],utraj[:,k])
    
        gx = q + A'*p[:,k+1]
        gu = r + B'*p[:,k+1]
    
        Gxx = Q + A'*P[:,:,k+1]*A + kron(p[:,k+1]',I(Nx))*comm(Nx,Nx)*Ax
        Guu = R + B'*P[:,:,k+1]*B + kron(p[:,k+1]',I(Nu))*comm(Nx,Nu)*Bu
        Gxu = A'*P[:,:,k+1]*B + kron(p[:,Nt]',I(Nx))*comm(Nx,Nx)*Au
        Gux = B'*P[:,:,k+1]*A + kron(p[:,Nt]',I(Nu))*comm(Nx,Nu)*Bx
    
        C = cholesky(Symmetric([Gxx Gxu; Gux Guu]), check=false)
        α = 1.0
        while !issuccess(C)
            Guu += α*I
            Gxx += α*I
            C = cholesky(Symmetric([Gxx Gxu; Gux Guu]), check=false)
            α = 10.0*α
        end
    
        j[:,k] .= Guu\gu
        K[:,:,k] .= Guu\Gux
    
        p[:,k] .= gx - K[:,:,k]'*gu + K[:,:,k]'*Guu*j[:,k] - Gxu*j[:,k]
        P[:,:,k] .= Gxx + K[:,:,k]'*Guu*K[:,:,k] - Gxu*K[:,:,k] - K[:,:,k]'*Gux
    
        ΔJ += gu'*j[:,k]
    end
    display(j)

    #Forward rollout with line search
    xn = zeros(Nx,Nt)
    un = zeros(Nu,Nt-1)
    xn[:,1] = xtraj[:,1]
    α = 1.0

    for k = 1:Nt-1
        un[:,k] .= utraj[:,k] - α*j[:,k] - K[:,:,k]*(xn[:,k]-xtraj[:,k])
        xn[:,k+1] .= quad_dynamics_rk4(xn[:,k],un[:,k])
    end
    Jn = cost(xn,un)
    
    while Jn > (J - 1e-2*α*ΔJ)
        α = 0.5*α
        for k = 1:Nt-1
            un[:,k] .= utraj[:,k] - α*j[:,k] - K[:,:,k]*(xn[:,k]-xtraj[:,k])
            xn[:,k+1] .= quad_dynamics_rk4(xn[:,k],un[:,k])
        end
        Jn = cost(xn,un)
    end
    display(α)
    
    J = Jn
    xtraj .= xn
    utraj .= un
end

2×60 Array{Float64,2}:
  0.276789   0.231704   1.63459  …  -0.247756  -0.197595  -0.0760077
 -0.107451  -0.0583631  0.15475      0.248686   0.198475   0.0763627

1.0

2×60 Array{Float64,2}:
  1.21297    0.255573   0.212515   …  -0.29987   -0.175469     0.0381024
 -0.164034  -0.137417  -0.0942177      0.105942  -0.00241665  -0.201056

1.0

2×60 Array{Float64,2}:
  0.664664   0.113022    0.827947  …  -0.364577  -0.174679   0.844091
 -0.221232  -0.0634397  -0.324556      0.3705     0.205428  -0.357579

...

1.0

2×60 Array{Float64,2}:
 0.0111137  0.00815443  0.00272503  -0.00317325  …  -1.70385e-6  -6.05896e-9
 0.0072053  0.00932186  0.00752123   0.0037711      -1.68265e-6  -6.06028e-9

1.0

2×60 Array{Float64,2}:
 -0.00163705  -0.00141327  -0.000852128  …  3.67631e-7  -2.74507e-10
 -0.00277831  -0.00455912  -0.00476127      3.60181e-7  -2.74663e-10

1.0

In [59]:
iter

28

In [19]:
#Set up visualization
using MeshCat
using RobotZoo: Quadrotor, PlanarQuadrotor
using CoordinateTransformations, Rotations, Colors, StaticArrays, RobotDynamics

function set_mesh!(vis, model::L;
        scaling=1.0, color=colorant"black"
    ) where {L <: Union{Quadrotor, PlanarQuadrotor}} 
    # urdf_folder = joinpath(@__DIR__, "..", "data", "meshes")
    urdf_folder = @__DIR__
    # if scaling != 1.0
    #     quad_scaling = 0.085 * scaling
    obj = joinpath(urdf_folder, "quadrotor_scaled.obj")
    if scaling != 1.0
        error("Scaling not implemented after switching to MeshCat 0.12")
    end
    robot_obj = MeshFileGeometry(obj)
    mat = MeshPhongMaterial(color=color)
    setobject!(vis["robot"]["geom"], robot_obj, mat)
    if hasfield(L, :ned)
        model.ned && settransform!(vis["robot"]["geom"], LinearMap(RotX(pi)))
    end
end

function visualize!(vis, model::PlanarQuadrotor, x::StaticVector)
    py,pz = x[1], x[2]
    θ = x[3]
    settransform!(vis["robot"], compose(Translation(0,py,pz), LinearMap(RotX(-θ))))
end

function visualize!(vis, model, tf::Real, X)
    fps = Int(round((length(X)-1)/tf))
    anim = MeshCat.Animation(fps)
    n = state_dim(model)
    for (k,x) in enumerate(X)
        atframe(anim, k) do
            x = X[k]
            visualize!(vis, model, SVector{n}(x)) 
        end
    end
    setanimation!(vis, anim)
end

visualize! (generic function with 2 methods)

In [20]:
vis = Visualizer()
model = PlanarQuadrotor()
set_mesh!(vis, model)
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /Users/zac/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8700))

In [60]:
X1 = [SVector{6}(x) for x in eachcol(xtraj)];
visualize!(vis, model, thist[end], X1)